In [3]:
import os
from sqlalchemy import create_engine, text
from pandas import read_sql, read_csv

In [5]:
user = os.environ['user']
password = os.environ['password']
host = os.environ['host']
port = os.environ['port']
database = os.environ['database']


engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')


query_securities = """
select * from golden.securities s 
where ticket in ('AE38','GOOGL','MSFT','VIST','AL30','CEPU','GGAL','MIRG','SAMI')
"""

query_portfolios = """
select * from golden.portfolios p
where partition_date = '2024-11-09'
"""

with engine.connect() as conn:
    df_securities = read_sql(query_securities, conn)
    df_portfolios = read_sql(query_portfolios, conn)

print(df_securities)
print(df_portfolios)


   id_securitie ticket                         full_name securitie_type  \
0             1   AE38      Bono Rep. Argentina Usd S...           bono   
1            12   CEPU                    CENTRAL PUERTO         accion   
2            15   GGAL          GRUPO FINANCIERO GALICIA         accion   
3            16   MIRG                            MIRGOR         accion   
4            22   AL30  BONOS REP. ARG. USD STEP UP 2030           bono   
5            23   SAMI                        SAN MIGUEL         accion   
6            24  GOOGL              CEDEAR ALPHABET INC.         cedear   
7            25   MSFT      CEDEAR MICROSOFT CORPORATION         cedear   
8            26   VIST            CEDEAR VISTA OIL & GAS         cedear   

  financial_instrument_type dividend_yield par_value  
0                renta fija           None      None  
1            renta variable           None      None  
2            renta variable           None      None  
3            renta variable  

In [7]:
def _delete_stocks_with_partition_date_portfolio_id(partition_date, id_portfolio, engine):
        """Delete stocks with partition_date and id_portfolio"""
        query_to_delete = text(
            "DELETE FROM golden.stocks WHERE partition_date = :partition_date "
            "AND id_portfolio = :id_portfolio"
        ).bindparams(partition_date=partition_date, id_portfolio=int(id_portfolio))
        
        print('Deleting stocks')
        print(query_to_delete)
        with engine.connect() as conn:
            result = conn.execute(query_to_delete)
            print(f'{result.rowcount} rows deleted')
            # commit
            conn.commit()
        print('Stocks deleted')

_delete_stocks_with_partition_date_portfolio_id('2024-11-09', 2, engine)



Deleting stocks
DELETE FROM golden.stocks WHERE partition_date = :partition_date AND id_portfolio = :id_portfolio
18 rows deleted
Stocks deleted


In [12]:
df_new_portfolio = read_csv('../data/silver/bullma_portfolio_2024-11-09.csv')
portfolio_name = 'bullma'
portfolio_date = '2024-11-09'
df_new_portfolio

,Ticket,Nombre,Cantidad,Ultimo Precio,PPC,Total
0,AE38,Bono Soberano USD Ley ARG 2038,200.0,73590.0,74010.04,147180.0
1,AL30,BONOS REP. ARG. USD STEP UP 2030,500.0,76130.0,76728.63,380650.0
2,CEPU,CENTRAL PUERTO,81.0,1345.0,1206.83,108945.0
3,GGAL,GRUPO FINANCIERO GALICIA,36.0,6250.0,4776.74,225000.0
4,MIRG,MIRGOR,9.0,23375.0,19516.57,210375.0
5,SAMI,SAN MIGUEL,400.0,1015.0,979.83,406000.0
6,GOOGL,CEDEAR ALPHABET INC.,79.0,3555.0,3549.74,280845.0
7,MSFT,CEDEAR MICROSOFT CORPORATION,18.0,16275.0,16298.31,292950.0
8,VIST,CEDEAR VISTA OIL & GAS,12.0,18275.0,19724.97,219300.0


In [45]:

df_transforms = df_new_portfolio.copy()
df_transforms['partition_date'] = portfolio_date
df_transforms = df_transforms.merge(df_securities[['ticket', 'id_securitie']], left_on='Ticket', right_on='ticket', how='left')
df_transforms = df_transforms.merge(df_portfolios[['id_portfolio', 'partition_date']], left_on='partition_date', right_on='partition_date', how='left')

df_transforms = df_transforms.drop(columns=['Ticket', 'Nombre', 'Total', 'ticket'])

df_transforms = df_transforms.rename(columns={'Cantidad': 'quantity', 'Ultimo Precio': 'last_price', 'PPC': 'average_purchase_price'})

# calculo la columna profit_loss_percent
df_transforms['profit_loss_percent'] = (df_transforms['last_price'] - df_transforms['average_purchase_price']) / df_transforms['average_purchase_price']
df_transforms['profit_loss_percent'] = df_transforms['profit_loss_percent'] * 100
df_transforms['profit_loss_percent'] = df_transforms['profit_loss_percent'].round(2)

# genero las columnas dividend_yield, interest_paid y amortization_paid en 0
df_transforms['dividend_yield'] = 0
df_transforms['interest_paid'] = 0
df_transforms['amortization_paid'] = 0

cols = ['id_portfolio', 'id_securitie', 'average_purchase_price', 'quantity', 'last_price', 'profit_loss_percent', 'dividend_yield', 'interest_paid', 'amortization_paid', 'partition_date']

df_transforms = df_transforms[cols]

df_transforms

,id_portfolio,id_securitie,average_purchase_price,quantity,last_price,profit_loss_percent,dividend_yield,interest_paid,amortization_paid,partition_date
0,2,23,74010.04,200.0,73590.0,-0.57,0,0,0,2024-11-09
1,2,32,76728.63,500.0,76130.0,-0.78,0,0,0,2024-11-09
2,2,33,1206.83,81.0,1345.0,11.45,0,0,0,2024-11-09
3,2,34,4776.74,36.0,6250.0,30.84,0,0,0,2024-11-09
4,2,35,19516.57,9.0,23375.0,19.77,0,0,0,2024-11-09
5,2,36,979.83,400.0,1015.0,3.59,0,0,0,2024-11-09
6,2,29,3549.74,79.0,3555.0,0.15,0,0,0,2024-11-09
7,2,30,16298.31,18.0,16275.0,-0.14,0,0,0,2024-11-09
8,2,31,19724.97,12.0,18275.0,-7.35,0,0,0,2024-11-09
